In [5]:
import pandas as pd
import numpy as np

In [6]:
def calculate_single_cm_to_m3(pump):
    inflow_df = pd.read_csv(f"../processed/pump_in_flow_appr_Helftheuvel.csv")
    
    # Calculate the difference in pump water volume, so calculating how much water causes the difference     
    inflow_df['in_minus_out'] = abs(inflow_df['flow_in'] - inflow_df['hstWaarde'])
    inflow_df['level'] = inflow_df.iloc[:,1].round()
    inflow_df['prev_level'] = inflow_df['level'].shift(1)


    # Creating a n x m dataframe for the data     
    indexes = np.array(inflow_df['level'].min() + np.arange(inflow_df['level'].max() - inflow_df['level'].min()))
    columns = np.array(inflow_df['level'].min() + np.arange(inflow_df['level'].max() - inflow_df['level'].min()))
    df = pd.DataFrame(0, index=indexes, columns=columns)

    # We fill this dataframe with the mean of all corresponding data belonging to every single cell     
    mean_df = inflow_df.groupby(['level','prev_level']).mean().reset_index()
    mean_df['rounded_level_diff'] = abs(mean_df['level'] - mean_df['prev_level'])
    mean_df.loc[mean_df['rounded_level_diff'] == 0, 'rounded_level_diff'] = 1
    mean_df = mean_df.loc[mean_df['level'] != mean_df['prev_level']]
    mean_df['mean_in_minus_out_per_cm'] = mean_df['in_minus_out'] / mean_df['rounded_level_diff']
    mean_df = mean_df.groupby(['prev_level']).mean().reset_index()
    resulting_df = pd.DataFrame(columns=['cm', 'm3'])
    resulting_df['cm'] = mean_df['prev_level']
    resulting_df['m3'] = mean_df['mean_in_minus_out_per_cm'].fillna(0).cumsum()
    print("Pump done!")
    return resulting_df


In [9]:
pumps = ['Engelerschans', 'Maaspoort', 'Rompert', 'Oude Engelenseweg', 'Helftheuvel']
for pump in pumps:
    pump_df = calculate_single_cm_to_m3(pump)
    pump_df.to_csv(f"../processed/{pump}_single_cm_m3.csv")

<ipython-input-6-b42bd1ec8d44>:16: RuntimeWarning: invalid value encountered in double_scalars
  new_value = df.loc[df.index == index, str(column)].iloc[0] / column_index_diff
<ipython-input-6-b42bd1ec8d44>:16: RuntimeWarning: invalid value encountered in true_divide
  new_value = df.loc[df.index == index, str(column)].iloc[0] / column_index_diff


Pump done!
Pump done!
Pump done!
